<a href="https://colab.research.google.com/github/DoItSon/playdata/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D(%EB%8B%A4%EC%A4%91%EB%AA%A8%EB%8D%B8)_%ED%95%B4%EB%B3%B4%EA%B8%B0!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 메모장
- mae score 넣기!

- 추후 베이지안 최저화를 통해 최적의 파라미터 찾기!

In [182]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score, r2_score, mean_absolute_error

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [183]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [184]:
import optuna

In [185]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [186]:
DATA_PATH = "/content/drive/MyDrive/"

In [187]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

In [188]:
categorical_features_enc = ['COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features_enc = ['ANONYMOUS_1', 'SAMPLE_TRANSFER_DAY', 'ANONYMOUS_2', 'AG', 'AL', 'B',
       'BA', 'BE', 'CA', 'CO', 'CR', 'CU', 'FE', 'H2O', 'LI', 'MG', 'MN', 'MO',
       'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'TI', 'V',
       'V40', 'ZN', 'CD', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4',
       'FTBN', 'FUEL', 'K', 'SOOTPERCENTAGE', 'U100', 'U75', 'U50', 'U25',
       'U20', 'U14', 'U6', 'U4', 'V100', 'COMPONENT_ARBITRARY_COMPONENT1',
       'COMPONENT_ARBITRARY_COMPONENT2', 'COMPONENT_ARBITRARY_COMPONENT3',
       'COMPONENT_ARBITRARY_COMPONENT4', 'YEAR_2007', 'YEAR_2008', 'YEAR_2009',
       'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015', 'YEAR_2016', 'YEAR_2017', 'YEAR_2018', 'YEAR_2019',
       'YEAR_2020', 'YEAR_2021', 'YEAR_2022']

## 전처리

In [189]:
CFG = {
    'n_splits':5,
    'SEED':42
}

- 내가 직접한 결측치

In [190]:
cols = ["CD","FH2O","FNOX","FOPTIMETHGLY","FOXID","FSO4","FTBN","FUEL","K","SOOTPERCENTAGE","U100","U75","U50","U25","U20","U14","U6","U4","V100"]

In [191]:
imputer = IterativeImputer(estimator =  GradientBoostingRegressor(),random_state=42)
tmp = imputer.fit_transform(train[cols])

In [192]:
x2 = pd.DataFrame(tmp,columns=cols)
train = train.drop(columns=cols)
train = pd.concat([train,x2],axis=1)
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,SOOTPERCENTAGE,U100,U75,U50,U25,U20,U14,U6,U4,V100
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,0.335832,0.000072,0.132080,0.717211,12.759819,17.317286,73.797220,2868.015566,41038.616096,14.911851
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,0.335832,0.000000,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,16.885307
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.332660,0.000000,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,16.727568
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,0.336773,0.000072,0.283716,0.484008,15.060832,29.698284,221.443845,7025.266402,21323.224450,14.982680
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,0.327862,0.000072,0.125349,31.862881,36.825836,66.418039,239.141792,6580.659555,18206.497105,13.260724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,0.334793,0.000072,0.107919,0.484008,18.883272,41.563347,233.039486,7112.017362,18852.246031,14.548606
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,0.400000,2.007214,2.780486,38.884917,21.979034,56.924185,366.228594,11770.663126,25678.055961,14.500000
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,0.336773,0.000072,0.283716,0.484008,15.060832,29.698284,221.443845,7025.266402,21323.224450,14.982680
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,0.334609,0.000000,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,17.743149


In [193]:
# fill_mean_func = lambda x: x.fillna(x.median())
# zx = train.groupby(["SAMPLE_TRANSFER_DAY","COMPONENT_ARBITRARY"]).apply(fill_mean_func)
# xc = zx.groupby(["ANONYMOUS_2","YEAR"]).apply(fill_mean_func)

# xc1 = xc.groupby(["ANONYMOUS_1","ANONYMOUS_2"]).apply(fill_mean_func)

# feat = pd.DataFrame(train["ID"].unique(),columns = ["ID"])

# xc2 = xc1.groupby(["MO"]).apply(fill_mean_func)

# features = feat.merge(xc2,how="left",on="ID")

# train = features

In [194]:
# train.isnull().sum()

- 다 0으로 채우기!

In [195]:
# train = train.fillna(0)
# test = test.fillna(0)

In [196]:
train.isnull().sum().sum()

0

## 데이터 분리

In [197]:
train_X  = train.drop(['ID', 'Y_LABEL'], axis = 1) # 정답값 제외
target  = train['Y_LABEL'] # 정답

test = test.drop(['ID'], axis = 1) # 추론 데이터

In [198]:
train_X

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,...,SOOTPERCENTAGE,U100,U75,U50,U25,U20,U14,U6,U4,V100
0,COMPONENT3,1486,2011,7,200,0,3,93,0,0,...,0.335832,0.000072,0.132080,0.717211,12.759819,17.317286,73.797220,2868.015566,41038.616096,14.911851
1,COMPONENT2,1350,2021,51,375,0,2,19,0,0,...,0.335832,0.000000,0.000000,1.000000,2.000000,4.000000,6.000000,216.000000,1454.000000,16.885307
2,COMPONENT2,2415,2015,2,200,0,110,1,1,0,...,0.332660,0.000000,0.000000,0.000000,0.000000,3.000000,39.000000,11261.000000,41081.000000,16.727568
3,COMPONENT3,7389,2010,2,200,0,8,3,0,0,...,0.336773,0.000072,0.283716,0.484008,15.060832,29.698284,221.443845,7025.266402,21323.224450,14.982680
4,COMPONENT3,3954,2015,4,200,0,1,157,0,0,...,0.327862,0.000072,0.125349,31.862881,36.825836,66.418039,239.141792,6580.659555,18206.497105,13.260724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,...,0.334793,0.000072,0.107919,0.484008,18.883272,41.563347,233.039486,7112.017362,18852.246031,14.548606
14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,...,0.400000,2.007214,2.780486,38.884917,21.979034,56.924185,366.228594,11770.663126,25678.055961,14.500000
14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,...,0.336773,0.000072,0.283716,0.484008,15.060832,29.698284,221.443845,7025.266402,21323.224450,14.982680
14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,...,0.334609,0.000000,0.000000,0.000000,7.000000,8.000000,100.000000,1625.000000,18890.000000,17.743149


# 원핫인코딩

In [199]:
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_X[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
train_X = pd.concat([train_X,tmp],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

tmp1 = pd.DataFrame(
    enc.transform(test[["COMPONENT_ARBITRARY",'YEAR']]).toarray(),
    columns = enc.get_feature_names_out()
)
test = pd.concat([test,tmp1],axis=1).drop(columns = ["COMPONENT_ARBITRARY",'YEAR'])

In [200]:
train_X.shape,target.shape,test.shape

((14095, 70), (14095,), (6041, 36))

# 스케일링

In [201]:
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features_enc:
        scaler = StandardScaler()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))

In [202]:
test.columns

Index(['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022'],
      dtype='object')

## 피처 추가

In [203]:
features = train_X[["FNOX","FH2O"]].sum(axis=1).reset_index()
features.columns = ["Index","FNOX+FH2O"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,...,YEAR_2014,YEAR_2015,YEAR_2016,YEAR_2017,YEAR_2018,YEAR_2019,YEAR_2020,YEAR_2021,YEAR_2022,FNOX+FH2O
0,-0.393763,-0.051413,-0.340760,-0.150214,-0.111628,0.281646,-0.238453,-0.041491,1.141962,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.942593
1,-0.426022,3.715319,-0.022576,-0.150214,-0.123127,-0.437686,-0.238453,-0.041491,1.087302,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.235567
2,-0.173409,-0.479451,-0.340760,-0.150214,1.118753,-0.612659,0.105735,-0.041491,-0.910846,-0.089633,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.349626
3,1.006399,-0.479451,-0.340760,-0.150214,-0.054133,-0.593217,-0.238453,-0.041491,0.400333,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880
4,0.191634,-0.308236,-0.340760,-0.150214,-0.134626,0.903771,-0.238453,-0.041491,-0.874406,-0.089633,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.136956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.362928,0.034194,-0.340760,-0.150214,-0.123127,1.331482,0.105735,-0.041491,-0.918269,-0.089633,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.050463
14091,-0.085884,-0.479451,-0.340760,-0.150214,-0.111628,0.203880,-0.238453,-0.041491,1.065033,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.950623
14092,-0.322130,0.119802,0.295608,-0.150214,-0.077131,-0.622379,0.105735,-0.041491,-0.913545,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880
14093,-0.153722,0.975877,0.295608,-0.150214,-0.123127,-0.583497,-0.238453,-0.041491,0.591983,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257839


In [204]:
features = train_X[["FNOX","FOXID"]].sum(axis=1).reset_index()
features.columns = ["Index","FNOX+FOXID"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,...,YEAR_2015,YEAR_2016,YEAR_2017,YEAR_2018,YEAR_2019,YEAR_2020,YEAR_2021,YEAR_2022,FNOX+FH2O,FNOX+FOXID
0,-0.393763,-0.051413,-0.340760,-0.150214,-0.111628,0.281646,-0.238453,-0.041491,1.141962,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.942593,-1.230859
1,-0.426022,3.715319,-0.022576,-0.150214,-0.123127,-0.437686,-0.238453,-0.041491,1.087302,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.235567,-1.282518
2,-0.173409,-0.479451,-0.340760,-0.150214,1.118753,-0.612659,0.105735,-0.041491,-0.910846,-0.089633,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.349626,0.119506
3,1.006399,-0.479451,-0.340760,-0.150214,-0.054133,-0.593217,-0.238453,-0.041491,0.400333,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880,-0.279892
4,0.191634,-0.308236,-0.340760,-0.150214,-0.134626,0.903771,-0.238453,-0.041491,-0.874406,-0.089633,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.136956,-0.901960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.362928,0.034194,-0.340760,-0.150214,-0.123127,1.331482,0.105735,-0.041491,-0.918269,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.050463,0.055176
14091,-0.085884,-0.479451,-0.340760,-0.150214,-0.111628,0.203880,-0.238453,-0.041491,1.065033,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.950623,3.394285
14092,-0.322130,0.119802,0.295608,-0.150214,-0.077131,-0.622379,0.105735,-0.041491,-0.913545,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880,-0.279892
14093,-0.153722,0.975877,0.295608,-0.150214,-0.123127,-0.583497,-0.238453,-0.041491,0.591983,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257839,-0.735539


In [205]:
features = train_X[["PB","MG"]].sum(axis=1).reset_index()
features.columns = ["Index","V100+V40"]

x = pd.concat([train_X,features],axis = 1)
train_X= x.drop(columns=["Index"])
train_X

,ANONYMOUS_1,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CO,...,YEAR_2016,YEAR_2017,YEAR_2018,YEAR_2019,YEAR_2020,YEAR_2021,YEAR_2022,FNOX+FH2O,FNOX+FOXID,V100+V40
0,-0.393763,-0.051413,-0.340760,-0.150214,-0.111628,0.281646,-0.238453,-0.041491,1.141962,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.942593,-1.230859,-0.327969
1,-0.426022,3.715319,-0.022576,-0.150214,-0.123127,-0.437686,-0.238453,-0.041491,1.087302,-0.089633,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.235567,-1.282518,-0.245714
2,-0.173409,-0.479451,-0.340760,-0.150214,1.118753,-0.612659,0.105735,-0.041491,-0.910846,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.349626,0.119506,-0.448117
3,1.006399,-0.479451,-0.340760,-0.150214,-0.054133,-0.593217,-0.238453,-0.041491,0.400333,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880,-0.279892,-0.316878
4,0.191634,-0.308236,-0.340760,-0.150214,-0.134626,0.903771,-0.238453,-0.041491,-0.874406,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.136956,-0.901960,-0.448117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.362928,0.034194,-0.340760,-0.150214,-0.123127,1.331482,0.105735,-0.041491,-0.918269,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.050463,0.055176,-0.448117
14091,-0.085884,-0.479451,-0.340760,-0.150214,-0.111628,0.203880,-0.238453,-0.041491,1.065033,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.950623,3.394285,-0.042254
14092,-0.322130,0.119802,0.295608,-0.150214,-0.077131,-0.622379,0.105735,-0.041491,-0.913545,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024880,-0.279892,-0.213894
14093,-0.153722,0.975877,0.295608,-0.150214,-0.123127,-0.583497,-0.238453,-0.041491,0.591983,-0.089633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.257839,-0.735539,-0.351206


## hp

In [206]:
cv = KFold(n_splits=5,shuffle=True,random_state=CFG["SEED"])

In [208]:
# def objective_1(trial):
#     hp = {
#         "max_depth" : trial.suggest_int("max_depth",2,15), # 찾을 범위의 최소값과 최대값
#         "min_samples_split" : trial.suggest_int("min_samples_split",2,10),
#         "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
#         "max_leaf_nodes" : trial.suggest_int("max_leaf_nodes",5,10),
#         "n_estimators" : trial.suggest_int("n_estimators",10,500),
#         "learning_rate" : trial.suggest_float("learning_rate",0.01,0.1)
#     }
#     cv = KFold(n_splits=5,shuffle=True,random_state=CFG['SEED'])
#     model = XGBClassifier(random_state=CFG['SEED'],**hp)

#     scores = cross_val_score(model, train_X, target,cv=cv, scoring="f1_macro") 
#     return np.mean(scores)

In [ ]:
# sampler = optuna.samplers.TPESampler(seed = CFG['SEED']) 

# study_1 = optuna.create_study(
#     direction= "maximize", 
#     sampler = sampler
# )

# study_1.optimize(objective_1,n_trials=50)

In [ ]:
print("Best Score:",study_1.best_value) 
print("Best hp", study_1.best_params)

In [164]:
study_1.best_params # 따로 적어놓기

NameError: ignored

In [162]:
LGC_hp = 

SyntaxError: ignored

In [210]:
XGC_hp = {'max_depth': 13, 'min_samples_split': 4, 'criterion': 'entropy', 'max_leaf_nodes': 5, 'n_estimators': 440, 'learning_rate': 0.08098309636582177}

## Classifier

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [212]:
model_list = [
    XGBClassifier(random_state=CFG["SEED"],**XGC_hp), 
    GradientBoostingClassifier(random_state=CFG["SEED"]),
    RandomForestClassifier(random_state=CFG["SEED"]),
    LGBMClassifier(random_state=CFG["SEED"])
]
for model in tqdm(model_list):
    scores = cross_val_score(model, train_X, target,cv = cv, scoring="f1_macro")
    print(scores.mean())

  0%|          | 0/4 [00:00<?, ?it/s]

0.8211534269319198
0.8157718772837871
0.8098884211959522
0.8199052668552653


In [ ]:
# 0.8177248066222159
# 0.8157718772837871
# 0.8098884211959522
# 0.8199052668552653
# 피처 추가

In [213]:
pred_list = []
for model in tqdm(model_list):
    model.fit(train_X,target)
    pred = model.predict_proba(train_X)[:,1]
    pred_list.append(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

In [214]:
target_X = pred_list
target_X

[array([1.4543427e-04, 9.2696759e-04, 9.9948418e-01, ..., 8.6470228e-03,
        1.9852633e-03, 7.4779493e-04], dtype=float32),
 array([0.015217  , 0.01820765, 0.98469769, ..., 0.16490377, 0.08045291,
        0.01632255]),
 array([0.  , 0.  , 0.91, ..., 0.1 , 0.01, 0.06]),
 array([0.0025238 , 0.00365661, 0.99875298, ..., 0.11187526, 0.0335154 ,
        0.00418149])]

- train의 부분 X의 예측값을 설명하는 Regressor

In [215]:
test_stage_features = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN',
       'COMPONENT_ARBITRARY_COMPONENT1', 'COMPONENT_ARBITRARY_COMPONENT2',
       'COMPONENT_ARBITRARY_COMPONENT3', 'COMPONENT_ARBITRARY_COMPONENT4',
       'YEAR_2007', 'YEAR_2008', 'YEAR_2009', 'YEAR_2010', 'YEAR_2011',
       'YEAR_2012', 'YEAR_2013', 'YEAR_2014', 'YEAR_2015', 'YEAR_2016',
       'YEAR_2017', 'YEAR_2018', 'YEAR_2019', 'YEAR_2020', 'YEAR_2021',
       'YEAR_2022']

In [216]:
train_reg = train_X[test_stage_features]
train_reg.shape,target_X[0].shape

((14095, 36), (14095,))

In [217]:
predict_list = []
model_list2 = [
    xgb.XGBRegressor(random_state=CFG["SEED"]), 
    GradientBoostingRegressor(random_state=CFG["SEED"]),
    RandomForestRegressor(random_state=CFG["SEED"]),
    lgb.LGBMRegressor(random_state=CFG["SEED"])
]
for i,model in enumerate(tqdm(model_list2)):
    scores = -1*cross_val_score(model, train_reg, target_X[i],cv = cv, scoring="neg_mean_absolute_error")
    print(scores.mean())

  0%|          | 0/4 [00:00<?, ?it/s]

[14:39:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.14697117507457733
0.08844481594390162
0.12579708407236603
0.11039728467850356


In [218]:
for i,models in enumerate(tqdm(model_list2)):
    model = models
    model.fit(train_reg,target_X[i]) 

    pred = model.predict(test)
    predict_list.append(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

[14:41:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [219]:
predict_list

[array([0.06253234, 0.10632381, 0.11387882, ..., 0.11019218, 0.04908064,
        0.03129789], dtype=float32),
 array([0.05807687, 0.11073513, 0.13019076, ..., 0.10128985, 0.06059254,
        0.04096563]),
 array([0.0164, 0.0373, 0.14  , ..., 0.2766, 0.0659, 0.0954]),
 array([0.04299842, 0.07302667, 0.12468164, ..., 0.10155261, 0.05881555,
        0.04622881])]

In [220]:
pred = np.mean(pred_list,axis=0)
pred.shape

(14095,)

In [221]:
y_pred = pred

In [222]:
y_pred

array([0.00447156, 0.00569781, 0.97323371, ..., 0.09635651, 0.03148839,
       0.02031296])

In [223]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
submit.Y_LABEL = pd.Series(np.where(y_pred > 0.35, 1, 0))

In [224]:
submit

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,1
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,1
6038,TEST_6038,0
6039,TEST_6039,0


In [225]:
submit.Y_LABEL.value_counts()

0    5522
1     519
Name: Y_LABEL, dtype: int64

In [226]:
submit.to_csv('/content/drive/MyDrive/submit.csv', index=False)